<a href="https://colab.research.google.com/github/Hydenx2004/Convolutional-Q-Learning-for-Pac-Man/blob/main/Deep_Convolutional_Q_Learning_for_Pac_Man.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Q-Learning for Pac-Man

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.4 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=4347f87e66ff1f2ca86883e6113415f88b501ddfecab316c11c10272878c4575
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed

### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):
    """
    Convolutional Neural Network for processing visual input and producing action values.

    Attributes:
        seed (int): Random seed for reproducibility.
        conv1 (nn.Conv2d): First convolutional layer.
        bn1 (nn.BatchNorm2d): Batch normalization for the first convolutional layer.
        conv2 (nn.Conv2d): Second convolutional layer.
        bn2 (nn.BatchNorm2d): Batch normalization for the second convolutional layer.
        conv3 (nn.Conv2d): Third convolutional layer.
        bn3 (nn.BatchNorm2d): Batch normalization for the third convolutional layer.
        conv4 (nn.Conv2d): Fourth convolutional layer.
        bn4 (nn.BatchNorm2d): Batch normalization for the fourth convolutional layer.
        fc1 (nn.Linear): First fully connected layer.
        fc2 (nn.Linear): Second fully connected layer.
        fc3 (nn.Linear): Output layer mapping to action values.
    """

    def __init__(self, action_size, seed=42):
        """
        Initialize the network.

        Args:
            action_size (int): Number of possible actions.
            seed (int): Random seed for reproducibility.
        """
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)

        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.bn4 = nn.BatchNorm2d(128)

        # Define fully connected layers
        self.fc1 = nn.Linear(10 * 10 * 128, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, action_size)

    def forward(self, state):
        """
        Forward pass through the network.

        Args:
            state (torch.Tensor): Input state (e.g., an image).

        Returns:
            torch.Tensor: Output action values.
        """
        # Apply first convolutional layer with ReLU activation and batch normalization
        x = F.relu(self.bn1(self.conv1(state)))

        # Apply second convolutional layer with ReLU activation and batch normalization
        x = F.relu(self.bn2(self.conv2(x)))

        # Apply third convolutional layer with ReLU activation and batch normalization
        x = F.relu(self.bn3(self.conv3(x)))

        # Apply fourth convolutional layer with ReLU activation and batch normalization
        x = F.relu(self.bn4(self.conv4(x)))

        # Flatten the tensor for fully connected layers
        x = x.view(x.size(0), -1)

        # Apply first fully connected layer with ReLU activation
        x = F.relu(self.fc1(x))

        # Apply second fully connected layer with ReLU activation
        x = F.relu(self.fc2(x))

        # Output layer for action values
        return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [ ]:
import gymnasium as gym
# Create the Ms. Pac-Man environment with deterministic settings
env = gym.make('MsPacmanDeterministic-v0', full_action_space=False)

# Get the shape of the observation space (state)
state_shape = env.observation_space.shape

# Get the size of the state (the first dimension of the state shape, assuming a 3D state representation)
state_size = env.observation_space.shape[0]

# Get the number of possible actions in the environment
number_actions = env.action_space.n

# Print the shape of the state, the size of the state, and the number of possible actions
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


State shape:  (210, 160, 3)
State size:  210
Number of actions:  9


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4
minibatch_size = 64
discount_factor = 0.99

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Preprocessing the frames

In [ ]:
from PIL import Image
from torchvision import transforms
def preprocess_frame(frame):
    """
    Preprocesses a given image frame by converting it to a PIL Image, applying transformations,
    and adding a batch dimension.

    Args:
    frame (numpy.ndarray): The input image frame represented as a NumPy array.

    Returns:
    torch.Tensor: The preprocessed image tensor with shape (1, C, H, W).
                  - 1: Batch size
                  - C: Number of channels (3 for RGB images, 1 for grayscale images)
                  - H: Height of the image (128 pixels)
                  - W: Width of the image (128 pixels)
    """

    # Convert the input NumPy array to a PIL Image object
    frame = Image.fromarray(frame)

    # Define a series of transformations to be applied to the image
    preprocess = transforms.Compose([
        transforms.Resize((128, 128)),  # Resize the image to 128x128 pixels
        transforms.ToTensor()           # Convert the image to a PyTorch tensor and normalize pixel values to [0, 1]
    ])

    # Apply the transformations to the image and add a batch dimension
    # The unsqueeze(0) method adds a new dimension at position 0 (batch dimension)
    return preprocess(frame).unsqueeze(0)

### Implementing the DCQN class

In [ ]:
class Agent():
    def __init__(self, action_size):
        """
        Initializes the agent with the given action size and sets up the neural networks,
        optimizer, and replay memory.

        Args:
        action_size (int): The number of possible actions the agent can take.
        """
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.action_size = action_size
        self.local_qnetwork = Network(action_size).to(self.device)
        self.target_qnetwork = Network(action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)
        self.memory = deque(maxlen=10000)

    def step(self, state, action, reward, next_state, done):
        """
        Adds the experience to the replay memory and triggers learning if the memory is sufficiently large.

        Args:
        state (numpy.ndarray): The current state.
        action (int): The action taken.
        reward (float): The reward received.
        next_state (numpy.ndarray): The next state.
        done (bool): Whether the episode has ended.
        """
        state = preprocess_frame(state)
        next_state = preprocess_frame(next_state)
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > minibatch_size:
            experiences = random.sample(self.memory, k=minibatch_size)
            self.learn(experiences, discount_factor)

    def act(self, state, epsilon=0.):
        """
        Selects an action using an epsilon-greedy policy.

        Args:
        state (numpy.ndarray): The current state.
        epsilon (float): The probability of selecting a random action (for exploration).

        Returns:
        int: The selected action.
        """
        state = preprocess_frame(state).to(self.device)
        self.local_qnetwork.eval()
        with torch.no_grad():
            action_values = self.local_qnetwork(state)
        self.local_qnetwork.train()
        if random.random() > epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, discount_factor):
        """
        Updates the Q-network based on a batch of experience tuples.

        Args:
        experiences (list): A batch of experience tuples.
        discount_factor (float): The discount factor (gamma).
        """
        states, actions, rewards, next_states, dones = zip(*experiences)
        states = torch.cat(states).float().to(self.device)
        actions = torch.tensor(actions).unsqueeze(1).long().to(self.device)
        rewards = torch.tensor(rewards).unsqueeze(1).float().to(self.device)
        next_states = torch.cat(next_states).float().to(self.device)
        dones = torch.tensor(dones).unsqueeze(1).float().to(self.device)

        # Compute the target Q-values
        next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
        q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))

        # Compute the expected Q-values
        q_expected = self.local_qnetwork(states).gather(1, actions)

        # Compute the loss
        loss = F.mse_loss(q_expected, q_targets)

        # Perform backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Initializing the DCQN agent

In [ ]:
agent = Agent(number_actions)

### Training the DCQN agent

In [ ]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 10000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 500.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'MsPacmanDeterministic-v0')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()